# 08 안전 운전자 예측
## 8.5 성능 개선 II: XGBoost 모델

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
data_path = '../../data/08_safe_driver/'

train_df = pd.read_csv(data_path+'train.csv', index_col='id')
test_df = pd.read_csv(data_path+'test.csv', index_col='id')
submission_df = pd.read_csv(data_path+'sample_submission.csv', index_col='id')

In [3]:
# 데이터 합치기
all_data = pd.concat([train_df, test_df], ignore_index=True)
all_data = all_data.drop(columns='target')
all_features = all_data.columns

In [4]:
# 명목형 피처 원핫 인코딩
from sklearn.preprocessing import OneHotEncoder
cat_features = [feature for feature in all_features if 'cat' in feature]
onehot_encoder = OneHotEncoder()
encoded_cat_matrix = onehot_encoder.fit_transform(all_data[cat_features])

In [5]:
# 파생 피처 추가
## 결측값 개수를 파생 피처로 생성
all_data['num_missing'] = (all_data==-1).sum(axis=1)

In [6]:
# 필요 없는 피처 제거
# drop_features = ['ps_ind_14', 'ps_ind_10_bin', 'ps_ind_11_bin', 'ps_ind_12_bin', 'ps_ind_13_bin', 'ps_car_14']
remaining_features = [feature for feature in all_features if ('cat' not in feature and 'calc' not in feature)] # and feature not in drop_features)]
remaining_features.append('num_missing')

In [7]:
## ind 분류의 피처 처리
ind_features = [feature for feature in all_features if 'ind' in feature]

is_first_feature = True
for ind_feature in ind_features:
    if is_first_feature:
        all_data['mix_ind'] = all_data[ind_feature].astype(str) + '_'
        is_first_feature=False
    else:
        all_data['mix_ind'] += all_data[ind_feature].astype(str) + '_'

In [8]:
cat_count_features = []
for feature in cat_features+['mix_ind']:
    val_counts_dict = all_data[feature].value_counts().to_dict()
    all_data[f'{feature}_count'] = all_data[feature].apply(lambda x: val_counts_dict[x])
    cat_count_features.append(f'{feature}_count')

In [9]:
# 필요없는 피처 제거
drop_features = ['ps_ind_14', 'ps_ind_10_bin', 'ps_ind_11_bin', 'ps_ind_12_bin', 'ps_ind_13_bin', 'ps_car_14']

all_data_remaining = all_data[remaining_features+cat_count_features].drop(columns=drop_features)
all_data_remaining.head()

,ps_ind_01,ps_ind_03,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_15,ps_ind_16_bin,ps_ind_17_bin,ps_ind_18_bin,...,ps_car_03_cat_count,ps_car_04_cat_count,ps_car_05_cat_count,ps_car_06_cat_count,ps_car_07_cat_count,ps_car_08_cat_count,ps_car_09_cat_count,ps_car_10_cat_count,ps_car_11_cat_count,mix_ind_count
0,2,5,0,1,0,0,11,0,1,0,...,1028142,1241334,431560,77845,1383070,249663,486510,1475460,18326,6
1,1,7,0,0,1,0,3,0,0,1,...,1028142,1241334,666910,329890,1383070,1238365,883326,1475460,12535,36
2,5,9,0,0,1,0,12,1,0,0,...,1028142,1241334,666910,147714,1383070,1238365,883326,1475460,19943,24
3,0,2,1,0,0,0,8,1,0,0,...,183044,1241334,431560,329890,1383070,1238365,36798,1475460,212989,2784
4,0,0,1,0,0,0,9,1,0,0,...,1028142,1241334,666910,147714,1383070,1238365,883326,1475460,26161,258


In [10]:
from scipy import sparse
all_data_sprs = sparse.hstack([
    sparse.csr_matrix(all_data_remaining),
    encoded_cat_matrix
], format='csr')

In [11]:
# 데이터 나누기
num_train = len(train_df)

X = all_data_sprs[:num_train]
X_test = all_data_sprs[num_train:]
y = train_df['target'].values

In [12]:
# 데이터셋 준비
import xgboost as xgb
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=0)

bayes_dtrain = xgb.DMatrix(X_train, y_train)
bayes_dvalid = xgb.DMatrix(X_valid, y_valid)

In [13]:
param_bounds = {
    'max_depth': (4,8),
    'subsample': (0.6, 0.9),
    'colsample_bytree': (0.7, 1.0),
    'min_child_weight': (5, 7),
    'gamma': (8, 11),
    'reg_alpha': (7, 9),
    'reg_lambda': (1.1, 1.5),
    'scale_pos_weight': (1.4, 1.6)
}

fixed_params = {
    'objective': 'binary:logistic',
    'learning_rate': 0.02,
    'random_state': 1991
}

In [14]:
def eval_gini(y_true, y_pred):
    assert y_true.shape == y_pred.shape
    n_samples = y_true.shape[0]
    L_mid = np.linspace(1/n_samples, 1, n_samples) # 대각선 값

    # 예측값에 대한 지니계수
    pred_order = y_true[y_pred.argsort()]
    # 로렌츠 곡선
    L_pred = np.cumsum(pred_order) / np.sum(pred_order) # array / scalar = array
    G_pred = np.sum(L_mid - L_pred) # 예측값에 대한 지니계수

    # 예측이 완벽할 때 지니계수
    true_order = y_true[y_true.argsort()]
    L_true = np.cumsum(true_order) / np.sum(true_order) # 로렌츠 곡선
    G_true = np.sum(L_mid - L_true) # 예측이 완벽할 때의 지니계수

    # 정규화된 지니계수
    return G_pred / G_true

In [15]:
# XGBoost용 gini() 함수
def gini(preds, dtrain):
    labels = dtrain.get_label() # 데이터셋의 타깃값을 반환
    #    평가지표 이름       평가 점수     평가 점수가 높을 수록 좋은지 여부
    return 'gini',   eval_gini(labels, preds)

In [18]:
# 베이지안 최적화용 평가지표 계산 함수 작성
def eval_function(max_depth, subsample, colsample_bytree, min_child_weight, reg_alpha, gamma, reg_lambda, scale_pos_weight):
    params = {
        'max_depth': int(round(max_depth)),
        'subsample': subsample,
        'colsample_bytree': colsample_bytree,
        'min_child_weight': min_child_weight,
        'gamma': gamma,
        'reg_alpha': reg_alpha,
        'reg_lambda': reg_lambda,
        'scale_pos_weight': scale_pos_weight
    }
    params.update(fixed_params)

    print(f'하이퍼파라미터: {params}')

    xgb_model = xgb.train(
        params=params,
        dtrain=bayes_dtrain,
        num_boost_round=2000,
        evals=[(bayes_dvalid, 'bayes_dvalid')],
        maximize=True,
        feval=gini,
        early_stopping_rounds=200,
        verbose_eval=False
    )
    best_iter = xgb_model.best_iteration
    preds = xgb_model.predict(bayes_dvalid, iteration_range=(0, best_iter))
    gini_score = eval_gini(y_valid, preds)
    print(f'지니계수 : {gini_score}\n')

    return gini_score

In [19]:
# 최적화 수행
from bayes_opt import BayesianOptimization
optimizer = BayesianOptimization(
    f=eval_function,
    pbounds=param_bounds,
    random_state=0
)
optimizer.maximize(init_points=3, n_iter=6)

|   iter    |  target   | colsam... |   gamma   | max_depth | min_ch... | reg_alpha | reg_la... | scale_... | subsample |
-------------------------------------------------------------------------------------------------------------------------
하이퍼파라미터: {'max_depth': 6, 'subsample': 0.867531900234624, 'colsample_bytree': 0.8646440511781974, 'min_child_weight': 6.0897663659937935, 'gamma': 10.14556809911726, 'reg_alpha': 7.84730959867781, 'reg_lambda': 1.3583576452266626, 'scale_pos_weight': 1.4875174422525386, 'objective': 'binary:logistic', 'learning_rate': 0.02, 'random_state': 1991}


C:\Users\dukim\miniconda3\envs\musthave_mldl_problem_solving_strategy\lib\site-packages\xgboost\training.py:39: UserWarning: `feval` is deprecated, use `custom_metric` instead.  They have different behavior when custom objective is also used.See https://xgboost.readthedocs.io/en/latest/tutorials/custom_metric_obj.html for details on the `custom_metric`.
  warnings.warn(


지니계수 : 0.2852804659784522

| 1         | 0.2853    | 0.8646    | 10.15     | 6.411     | 6.09      | 7.847     | 1.358     | 1.488     | 0.8675    |
하이퍼파라미터: {'max_depth': 7, 'subsample': 0.6261387899104622, 'colsample_bytree': 0.9890988281503088, 'min_child_weight': 6.0577898395058085, 'gamma': 9.150324556477333, 'reg_alpha': 8.136089122187865, 'reg_lambda': 1.4702386553170643, 'scale_pos_weight': 1.4142072116395774, 'objective': 'binary:logistic', 'learning_rate': 0.02, 'random_state': 1991}
지니계수 : 0.2848088779310762

| 2         | 0.2848    | 0.9891    | 9.15      | 7.167     | 6.058     | 8.136     | 1.47      | 1.414     | 0.6261    |
하이퍼파라미터: {'max_depth': 7, 'subsample': 0.8341587528859367, 'colsample_bytree': 0.7060655192320977, 'min_child_weight': 6.7400242964936385, 'gamma': 10.497859536643814, 'reg_alpha': 8.957236684465528, 'reg_lambda': 1.4196634256866894, 'scale_pos_weight': 1.4922958724505864, 'objective': 'binary:logistic', 'learning_rate': 0.02, 'random_state': 1991}
지

C:\Users\dukim\miniconda3\envs\musthave_mldl_problem_solving_strategy\lib\site-packages\xgboost\training.py:39: UserWarning: `feval` is deprecated, use `custom_metric` instead.  They have different behavior when custom objective is also used.See https://xgboost.readthedocs.io/en/latest/tutorials/custom_metric_obj.html for details on the `custom_metric`.
  warnings.warn(


지니계수 : 0.28372532851421195

| 4         | 0.2837    | 0.8843    | 10.45     | 6.838     | 6.494     | 8.552     | 1.381     | 1.423     | 0.7002    |
하이퍼파라미터: {'max_depth': 7, 'subsample': 0.8535233675350644, 'colsample_bytree': 0.92975858050776, 'min_child_weight': 6.249564429359247, 'gamma': 9.95563546750357, 'reg_alpha': 8.411512219837842, 'reg_lambda': 1.424460008293778, 'scale_pos_weight': 1.5416807226581535, 'objective': 'binary:logistic', 'learning_rate': 0.02, 'random_state': 1991}


...

지니계수 : 0.28467572646560496

| 7         | 0.2847    | 0.8818    | 9.013     | 6.927     | 6.992     | 7.641     | 1.356     | 1.449     | 0.6931    |
하이퍼파라미터: {'max_depth': 5, 'subsample': 0.6261564417044092, 'colsample_bytree': 0.8763145220620449, 'min_child_weight': 5.135323353557588, 'gamma': 8.39495450163982, 'reg_alpha': 8.950443047087845, 'reg_lambda': 1.4235649099168255, 'scale_pos_weight': 1.5217625173811569, 'objective': 'binary:logistic', 'learning_rate': 0.02, 'random_state': 1991}


C:\Users\dukim\miniconda3\envs\musthave_mldl_problem_solving_strategy\lib\site-packages\xgboost\training.py:39: UserWarning: `feval` is deprecated, use `custom_metric` instead.  They have different behavior when custom objective is also used.See https://xgboost.readthedocs.io/en/latest/tutorials/custom_metric_obj.html for details on the `custom_metric`.
  warnings.warn(


지니계수 : 0.28457181017567373

| 8         | 0.2846    | 0.8763    | 8.395     | 4.561     | 5.135     | 8.95      | 1.424     | 1.522     | 0.6262    |
하이퍼파라미터: {'max_depth': 6, 'subsample': 0.857971740304964, 'colsample_bytree': 0.9583821245229369, 'min_child_weight': 6.158305055403563, 'gamma': 9.305332775334449, 'reg_alpha': 8.200928434091152, 'reg_lambda': 1.2571039588093065, 'scale_pos_weight': 1.4700266933495618, 'objective': 'binary:logistic', 'learning_rate': 0.02, 'random_state': 1991}


C:\Users\dukim\miniconda3\envs\musthave_mldl_problem_solving_strategy\lib\site-packages\xgboost\training.py:39: UserWarning: `feval` is deprecated, use `custom_metric` instead.  They have different behavior when custom objective is also used.See https://xgboost.readthedocs.io/en/latest/tutorials/custom_metric_obj.html for details on the `custom_metric`.
  warnings.warn(


지니계수 : 0.28538973194158385

| 9         | 0.2854    | 0.9584    | 9.305     | 5.594     | 6.158     | 8.201     | 1.257     | 1.47      | 0.858     |


In [20]:
# 결과 확인
max_params = optimizer.max['params']
max_params

{'colsample_bytree': 0.9583821245229369,
 'gamma': 9.305332775334449,
 'max_depth': 5.594282602920541,
 'min_child_weight': 6.158305055403563,
 'reg_alpha': 8.200928434091152,
 'reg_lambda': 1.2571039588093065,
 'scale_pos_weight': 1.4700266933495618,
 'subsample': 0.857971740304964}

In [21]:
# 정수형 하이퍼파라미터 변환
max_params['max_depth'] = int(round(max_params['max_depth']))
# 값이 고정된 파라미터 추가
max_params.update(fixed_params)
max_params

{'colsample_bytree': 0.9583821245229369,
 'gamma': 9.305332775334449,
 'max_depth': 6,
 'min_child_weight': 6.158305055403563,
 'reg_alpha': 8.200928434091152,
 'reg_lambda': 1.2571039588093065,
 'scale_pos_weight': 1.4700266933495618,
 'subsample': 0.857971740304964,
 'objective': 'binary:logistic',
 'learning_rate': 0.02,
 'random_state': 1991}

### 8.5.3 모델 훈련 및 성능 검증

In [22]:
# OOF 방식으로 LightGBM 훈련
from sklearn.model_selection import StratifiedKFold

folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=1991)

In [28]:
oof_val_preds = np.zeros(X.shape[0])
oof_test_preds = np.zeros(X_test.shape[0])
for idx, (train_idx, valid_idx) in enumerate(folds.split(X, y)):
    print('#'*40, f'폴드 {idx+1} / 폴드 {folds.n_splits}', '#'*40)
    X_train, y_train = X[train_idx], y[train_idx]
    X_valid, y_valid = X[valid_idx], y[valid_idx]

    dtrain = xgb.DMatrix(X_train, y_train)
    dvalid = xgb.DMatrix(X_valid, y_valid)
    dtest = xgb.DMatrix(X_test)

    xgb_model = xgb.train(
        params=max_params,             # 훈련용 하이퍼파라미터
        dtrain=bayes_dtrain,          # 훈련 데이터셋
        num_boost_round=2000,      # 부스팅 반복 횟수
        evals=[(dvalid, 'valid')],         # 상능 평가용 검증 데이터셋
        maximize=True,
        feval=gini,                # 검증용 평가지표
        early_stopping_rounds=200, # 조기종료 조건
        verbose_eval=100           # 100번째마다 점수 출력
    )
    best_iter = xgb_model.best_iteration
    oof_test_preds += xgb_model.predict(dtest)/folds.n_splits
    oof_val_preds[valid_idx] += xgb_model.predict(dvalid)

    gini_score = eval_gini(y_valid, oof_val_preds[valid_idx])
    print(f'폴드 {idx+1} 지니계수 : {gini_score}\n')

######################################## 폴드 1 / 폴드 5 ########################################


C:\Users\dukim\miniconda3\envs\musthave_mldl_problem_solving_strategy\lib\site-packages\xgboost\training.py:39: UserWarning: `feval` is deprecated, use `custom_metric` instead.  They have different behavior when custom objective is also used.See https://xgboost.readthedocs.io/en/latest/tutorials/custom_metric_obj.html for details on the `custom_metric`.
  warnings.warn(


[0]	valid-logloss:0.67671	valid-gini:0.18176
[100]	valid-logloss:0.19163	valid-gini:0.26620
[200]	valid-logloss:0.15769	valid-gini:0.30439
[300]	valid-logloss:0.15400	valid-gini:0.32071
[400]	valid-logloss:0.15317	valid-gini:0.33208
[500]	valid-logloss:0.15286	valid-gini:0.33811
[600]	valid-logloss:0.15264	valid-gini:0.34294
[700]	valid-logloss:0.15246	valid-gini:0.34775
[800]	valid-logloss:0.15230	valid-gini:0.35132
[900]	valid-logloss:0.15218	valid-gini:0.35437
[1000]	valid-logloss:0.15205	valid-gini:0.35755
[1100]	valid-logloss:0.15194	valid-gini:0.35998
[1200]	valid-logloss:0.15183	valid-gini:0.36271
[1300]	valid-logloss:0.15170	valid-gini:0.36551
[1400]	valid-logloss:0.15159	valid-gini:0.36781
[1500]	valid-logloss:0.15151	valid-gini:0.37008
[1600]	valid-logloss:0.15141	valid-gini:0.37226
[1700]	valid-logloss:0.15133	valid-gini:0.37434
[1800]	valid-logloss:0.15126	valid-gini:0.37590
[1900]	valid-logloss:0.15116	valid-gini:0.37828
[1999]	valid-logloss:0.15107	valid-gini:0.38025
폴드 1

In [29]:
print(f'OOF 검증 데이터 지니계수: {eval_gini(y, oof_val_preds)}')

OOF 검증 데이터 지니계수: 0.3672147450586398


In [30]:
submission_df['target'] = oof_test_preds
submission_df.to_csv(data_path+'submission_enhanced2.csv')